In [26]:
import sys, os, json, time
from collections import Counter

In [27]:
import pandas as pd
import numpy as np

In [28]:
with open('secret.conf', 'r') as f:
    for line in f:
        line = line.strip().split(' ')
        if line[0] == 'export':
            equal_sign_position = line[1].find('=')
            k = line[1][:equal_sign_position]
            v = line[1][equal_sign_position+1:].strip('"')
            os.environ[k] = v
            

In [29]:
from ms_academic import MicrosoftAcademic

In [30]:
api = MicrosoftAcademic()

In [31]:
api.attributes = 'Id,AA.AuId,AA.AuN,AA.AfN,AA.AfId'
api.query_threshold = 9e6

In [32]:
with open('coauthorship.json', 'r') as f:
    j = json.load(f)

In [33]:
x = j['nodes'][0]
x['author_name']

u'Daniel Jackson'

In [34]:
def query_author_affil(author_name, authors):
    r = api.interpret_and_evaluate(author_name)
    if author_name not in authors:
        authors[author_name] = {'author_id': Counter(), 'affil_name': Counter(), 'affil_id': Counter()}
    for item in r:
        for aa in item['AA']:
            if aa['AuN'] == author_name:
                found = True
                author_id = aa.get('AuId')
                affil_name = aa.get('AfN')
                affil_id = aa.get('AfId')
                authors[author_name]['author_id'][author_id] += 1
                authors[author_name]['affil_name'][affil_name] += 1
                authors[author_name]['affil_id'][affil_id] += 1

start = time.time()
authors = {}
none_found = []
errors = []
i = 0
sleep_dur = 10  # 10 seconds
sleep_interval = 10  # every 10 queries
for node in j['nodes']:
    author_name = node['author_name'].lower()
    found = False
    try:
        query_author_affil(author_name, authors)
    except Exception as e:
        errors.append( (i, author_name, e) )
        # try one more time
        print("error. sleeping {} seconds".format(sleep_dur))
        time.sleep(sleep_dur)
        try:
            query_author_affil(author_name, authors)
        except Exception as e:
            errors.append( (i, author_name, e) )
    if not found:
        none_found.append(author_name)
    i += 1
    if i % sleep_interval == 0:
        print("{} queries in {:.1f} seconds. sleeping {} seconds".format(i, time.time()-start, sleep_dur))
        time.sleep(sleep_dur)

error. sleeping 10 seconds
10 queries in 19.1 seconds. sleeping 10 seconds
20 queries in 33.8 seconds. sleeping 10 seconds
30 queries in 50.4 seconds. sleeping 10 seconds
40 queries in 68.9 seconds. sleeping 10 seconds
50 queries in 85.9 seconds. sleeping 10 seconds
60 queries in 100.9 seconds. sleeping 10 seconds
70 queries in 118.1 seconds. sleeping 10 seconds
80 queries in 137.2 seconds. sleeping 10 seconds
90 queries in 152.8 seconds. sleeping 10 seconds
100 queries in 168.6 seconds. sleeping 10 seconds
110 queries in 188.8 seconds. sleeping 10 seconds
120 queries in 207.6 seconds. sleeping 10 seconds
130 queries in 226.1 seconds. sleeping 10 seconds
140 queries in 244.8 seconds. sleeping 10 seconds
150 queries in 263.7 seconds. sleeping 10 seconds
160 queries in 281.4 seconds. sleeping 10 seconds
170 queries in 298.8 seconds. sleeping 10 seconds
180 queries in 316.5 seconds. sleeping 10 seconds
190 queries in 331.7 seconds. sleeping 10 seconds
200 queries in 347.4 seconds. sleepin

In [35]:
len(authors)

457

In [39]:
len(none_found)

457

In [40]:
errors

[(9, u'amyra meidiana', KeyError('expr')),
 (298, u'yaniv frishman', KeyError('expr'))]

In [38]:
import cPickle as pickle
with open('author_affil_mag.pickle', 'wb') as outf:
    pickle.dump(authors, outf)

In [45]:
rows = []
for node in j['nodes']:
    cl_bottom = node['cl_bottom']
    cl_top = node['cl_top']
    author_name = node['author_name']
    cl_depth = len(cl_bottom.split(':'))
    if author_name.lower() in authors:
        affil_name = authors[author_name.lower()]['affil_name']
        if affil_name:
            affil_name = affil_name.most_common()[0][0]
        else:
            affil_name = ''
        affil_id = authors[author_name.lower()]['affil_id']
        if affil_id:
            affil_id = affil_id.most_common()[0][0]
        else:
            affil_id = ''
        rows.append( (author_name, cl_bottom, cl_top, cl_depth, affil_name, affil_id) )
df = pd.DataFrame(rows, columns='author_name cl_bottom cl_top cl_depth affil_name affil_id'.split())

In [46]:
df[df.cl_depth>1].sort_values(['cl_depth', 'cl_bottom'], ascending=False)

,author_name,cl_bottom,cl_top,cl_depth,affil_name,affil_id
24,Takafumi Saito,2:1:2:2,2,4,yamagata university,112524849
136,Jun Sese,2:1:2:2,2,4,university of tokyo,74801974
177,Takayuki Itoh,2:1:2:2,2,4,ibm,1341412227
277,Aika Terada,2:1:2:2,2,4,tokyo institute of technology,114531698
318,Rina Nakazawa,2:1:2:2,2,4,ochanomizu university,26120043
40,Michael Ogawa,2:1:2:1,2,4,university of california davis,84218800
130,Chris Muelder,2:1:2:1,2,4,university of california davis,84218800
142,Kwan-Liu Ma,2:1:2:1,2,4,,
175,Diane Felmlee,2:1:2:1,2,4,,
215,Robert Faris,2:1:2:1,2,4,university of california davis,84218800


In [47]:
df[df.author_name.isin(['Aaron Barsky', 'Tamara Munzner'])]

,author_name,cl_bottom,cl_top,cl_depth,affil_name,affil_id
22,Aaron Barsky,2:1:1:4,2,4,university of british columbia,141945490
305,Tamara Munzner,2:1:1:2,2,4,university of british columbia,141945490


In [48]:
errors

[(9, u'amyra meidiana', KeyError('expr')),
 (298, u'yaniv frishman', KeyError('expr'))]

In [49]:
df.affil_name.value_counts()

                                                                    152
university of stuttgart                                               9
l abri                                                                7
monash university                                                     6
french institute for research in computer science and automation      6
centre national de la recherche scientifique                          5
university of tubingen                                                5
university of trier                                                   5
university of california davis                                        5
university of calgary                                                 5
bielefeld university                                                  5
university of maryland college park                                   5
graz university of technology                                         5
university of perugia                                           

In [50]:
df.affil_id.value_counts()

              152
100066346       9
872287936       7
1326498283      6
1290206253      6
56590836        6
89864525        5
20121455        5
1341412227      5
168635309       5
84218800        5
8087733         5
27483092        5
4092182         5
66946132        5
130701444       4
153267046       4
138006243       4
189712700       4
129604602       4
121934539       3
121883995       3
202697423       3
111088046       3
149899117       3
83019370        3
1303153112      3
1325886976      3
200769079       3
168864056       3
             ... 
55176711        1
44265643        1
114027177       1
149704539       1
1324840837      1
202367325       1
52357470        1
1304085615      1
17974374        1
87048295        1
90610280        1
130769515       1
219193219       1
160606119       1
139264467       1
895027328       1
70931966        1
1324220072      1
135310074       1
165779595       1
97018004        1
32394136        1
198357145       1
208215962       1
102149020 